<a href="https://colab.research.google.com/github/cff100/MecFLu2/blob/main/MecFlu2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# prompt: pip install openeo

!pip install openeo

In [28]:
import openeo

connection = openeo.connect("openeo.dataspace.copernicus.eu")

In [29]:
connection.authenticate_oidc()


Authenticated using refresh token.


<Connection to 'https://openeo.dataspace.copernicus.eu/openeo/1.2/' with OidcBearerAuth>

In [30]:
datacube = connection.load_collection(
    "SENTINEL2_L2A",
    spatial_extent={"west": 5.14, "south": 51.17, "east": 5.17, "north": 51.19},
    temporal_extent = ["2021-02-01", "2021-04-30"],
    bands=["B02", "B04", "B08"],
    max_cloud_cover=85,
)

In [31]:
datacube = datacube.max_time()

In [32]:
# While not necessary, it is also recommended to give your batch job a descriptive title so it’s easier to identify in your job listing.
job = datacube.execute_batch() # Use 'datacube' instead of 'cube'

0:00:00 Job 'j-2408152fbb374d758f4ed51914118000': send 'start'
0:00:12 Job 'j-2408152fbb374d758f4ed51914118000': created (progress 0%)
0:00:18 Job 'j-2408152fbb374d758f4ed51914118000': created (progress 0%)
0:00:24 Job 'j-2408152fbb374d758f4ed51914118000': running (progress N/A)
0:00:32 Job 'j-2408152fbb374d758f4ed51914118000': running (progress N/A)
0:00:42 Job 'j-2408152fbb374d758f4ed51914118000': running (progress N/A)
0:00:55 Job 'j-2408152fbb374d758f4ed51914118000': running (progress N/A)
0:01:10 Job 'j-2408152fbb374d758f4ed51914118000': running (progress N/A)
0:01:30 Job 'j-2408152fbb374d758f4ed51914118000': running (progress N/A)
0:01:54 Job 'j-2408152fbb374d758f4ed51914118000': running (progress N/A)
0:02:24 Job 'j-2408152fbb374d758f4ed51914118000': running (progress N/A)
0:03:02 Job 'j-2408152fbb374d758f4ed51914118000': finished (progress 100%)


In [37]:
import openeo

# First, we connect to the back-end and authenticate.
con = openeo.connect("openeo.dataspace.copernicus.eu")
con.authenticate_oidc()

# Now that we are connected, we can initialize our datacube object with the area of interest
# and the time range of interest using Sentinel 1 data.
datacube = connection.load_collection(
    "SENTINEL2_L2A",
    spatial_extent={"west": 5.14, "south": 51.17, "east": 5.17, "north": 51.19},
    temporal_extent = ["2021-02-01", "2021-04-30"],
    bands=["B02", "B04", "B08"],
    max_cloud_cover=85,
)

# By filtering as early as possible (directly in load_collection() in this case),
# we make sure the back-end only loads the data we are interested in and avoid incurring unneeded costs.


#From this data cube, we can now select the individual bands with the DataCube.band() method and rescale the digital number values to physical reflectances:
blue = datacube.band("B02") * 0.0001 # Use 'datacube' instead of 'sentinel2_cube'
red = datacube.band("B04") * 0.0001
nir = datacube.band("B08") * 0.0001


# We now want to compute the enhanced vegetation index and can do that directly with these band variables:
evi_cube = 2.5 * (nir - red) / (nir + 6.0 * red - 7.5 * blue + 1.0)

# Load the data cube containing the "SCL" band. Make sure to adjust parameters as needed.
s2_scl = connection.load_collection(
    "SENTINEL2_L2A",
    spatial_extent={"west": 5.14, "south": 51.17, "east": 5.17, "north": 51.19},
    temporal_extent=["2021-02-01", "2021-04-30"],
    bands=["SCL"],
    max_cloud_cover=85
)

# Now we can use the compact “band math” feature again to build a binary mask with a simple comparison operation:
# Select the "SCL" band from the data cube
scl_band = s2_scl.band("SCL")
# Build mask to mask out everything but class 4 (vegetation)
mask = (scl_band != 4)

# Before we can apply this mask to the EVI cube we have to resample it, as the “SCL” layer has a “ground sample distance” of 20 meter, while it is 10 meter for the “B02”, “B04” and “B08” bands. We can easily do the resampling by referring directly to the EVI cube.
mask_resampled = mask.resample_cube_spatial(evi_cube)

# Apply the mask to the `evi_cube`
evi_cube_masked = evi_cube.mask(mask_resampled)

# Because GeoTIFF does not support a temporal dimension, we first eliminate it by taking the temporal maximum value for each pixel:
evi_composite = evi_cube.max_time()

# Now we can download this to a local file:
evi_composite.download("evi-composite.tiff")



Authenticated using refresh token.
